# Project: Bangla Dialect-to-Standard Normalization
## Phase 2A: Robust Automated Draft Transcription (P100 Optimized)

**Optimization:** Uses 'Medium' model + Aggressive Memory Management to prevent Kaggle Kernel Freezes.

---

### Abstract
This notebook generates draft transcriptions for the segmented dialect audio. It is specifically optimized for the Kaggle P100 GPU environment by using the Whisper 'Medium' architecture and implementing incremental checkpoint saving. This ensures that even if the kernel times out, progress is preserved.


In [1]:
# CELL 1: Environment Setup (CORRECTED)
# Installs Whisper FIRST, then imports it.

!pip install -q openai-whisper

import os
import gc
import logging
import warnings
import pandas as pd
import torch
import whisper
from glob import glob
from tqdm.notebook import tqdm

# Suppress warnings
warnings.filterwarnings('ignore')

# Configure Logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s: %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger("Transcriber")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✓ Hardware Device: {DEVICE.upper()}", flush=True)

if DEVICE == "cpu":
    print("⚠️ WARNING: Running on CPU! This will be extremely slow.", flush=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 41.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
✓ Hardware Device: CUDA


In [2]:
# CELL 2: Robust Data Discovery
# Finds your segmented inventory and audio files automatically

def locate_data():
    # 1. Find CSV
    csv_files = glob("/kaggle/input/**/segmented_inventory.csv", recursive=True)
    if not csv_files:
        print("❌ Error: 'segmented_inventory.csv' not found. Please add Phase 1C output as input.", flush=True)
        return None, None
    
    csv_path = csv_files[0]
    print(f"✓ Found Inventory: {csv_path}", flush=True)
    
    # 2. Find Audio Directory
    df = pd.read_csv(csv_path)
    if df.empty:
        print("❌ Error: Inventory CSV is empty!", flush=True)
        return None, None

    # Check first file location
    sample_file = df.iloc[0]['filename']
    audio_files = glob(f"/kaggle/input/**/{sample_file}", recursive=True)
    
    if not audio_files:
        # Fallback for same-session
        if os.path.exists(f"/kaggle/working/segmented_dataset/{sample_file}"):
            return df, "/kaggle/working/segmented_dataset"
        print(f"❌ Error: Audio file '{sample_file}' not found.", flush=True)
        return df, None
    
    audio_dir = os.path.dirname(audio_files[0])
    print(f"✓ Found Audio Directory: {audio_dir}", flush=True)
    return df, audio_dir

df_inventory, AUDIO_DIR = locate_data()

✓ Found Inventory: /kaggle/input/phase-1c-part2-segmentation/manifests/segmented_inventory.csv
✓ Found Audio Directory: /kaggle/input/phase-1c-part2-segmentation/segmented_dataset


In [3]:
# CELL 3: Load Optimized Model (Medium)
# Using 'medium' ensures stability on P100

MODEL_SIZE = "medium"

if AUDIO_DIR:
    print(f"⏳ Loading Whisper '{MODEL_SIZE}' model...", flush=True)
    try:
        model = whisper.load_model(MODEL_SIZE, device=DEVICE)
        print("✓ Model Loaded Successfully.", flush=True)
    except Exception as e:
        print(f"❌ Model Load Failed: {e}", flush=True)
        raise

⏳ Loading Whisper 'medium' model...


100%|██████████████████████████████████████| 1.42G/1.42G [00:06<00:00, 232MiB/s]


✓ Model Loaded Successfully.


In [4]:
# CELL 4: Fail-Safe Transcription Loop

CHECKPOINT_PATH = "/kaggle/working/draft_transcriptions_checkpoint.csv"

if AUDIO_DIR and df_inventory is not None:
    results = []
    total_files = len(df_inventory)
    
    print(f"🚀 Starting Inference on {total_files} files...", flush=True)
    
    for idx, row in tqdm(df_inventory.iterrows(), total=total_files):
        file_name = row['filename']
        file_path = os.path.join(AUDIO_DIR, file_name)
        
        try:
            if not os.path.exists(file_path):
                continue

            # Transcribe (Beam size 1 for speed)
            result = model.transcribe(file_path, language="bn", beam_size=1)
            
            results.append({
                "filename": file_name,
                "dialect": row['dialect'],
                "duration": row['duration'],
                "machine_transcript": result['text'].strip(),
                "human_correction": "" 
            })
            
            # PROGRESS UPDATE (Crucial for Save Version Logs)
            if (idx + 1) % 20 == 0:
                print(f"   --> Processed {idx + 1}/{total_files} files...", flush=True)
                
                # Memory Cleanup
                torch.cuda.empty_cache()
                gc.collect()
            
            # Checkpoint Save (Every 50 files)
            if (idx + 1) % 50 == 0:
                pd.DataFrame(results).to_csv(CHECKPOINT_PATH, index=False)

        except Exception as e:
            print(f"⚠️ Error on {file_name}: {e}", flush=True)
            continue

    # Final Save
    final_path = "/kaggle/working/draft_transcriptions_FINAL.csv"
    pd.DataFrame(results).to_csv(final_path, index=False)
    print(f"✅ Job Complete. Saved to {final_path}", flush=True)
else:
    print("❌ Skipping transcription due to missing setup.", flush=True)

🚀 Starting Inference on 577 files...


  0%|          | 0/577 [00:00<?, ?it/s]

   --> Processed 20/577 files...
   --> Processed 40/577 files...
   --> Processed 60/577 files...
   --> Processed 80/577 files...
   --> Processed 100/577 files...
   --> Processed 120/577 files...
   --> Processed 140/577 files...
   --> Processed 160/577 files...
   --> Processed 180/577 files...
   --> Processed 200/577 files...
   --> Processed 220/577 files...
   --> Processed 240/577 files...
   --> Processed 260/577 files...
   --> Processed 280/577 files...
   --> Processed 300/577 files...
   --> Processed 320/577 files...
   --> Processed 340/577 files...
   --> Processed 360/577 files...
   --> Processed 380/577 files...
   --> Processed 400/577 files...
   --> Processed 420/577 files...
   --> Processed 440/577 files...
   --> Processed 460/577 files...
   --> Processed 480/577 files...
   --> Processed 500/577 files...
   --> Processed 520/577 files...
   --> Processed 540/577 files...
   --> Processed 560/577 files...
✅ Job Complete. Saved to /kaggle/working/draft_trans

In [5]:
# CELL 5: Verify Output
import os
import pandas as pd

out_file = "/kaggle/working/draft_transcriptions_FINAL.csv"
if os.path.exists(out_file):
    df = pd.read_csv(out_file)
    print(f"\n📄 Final File Generated: {out_file}")
    print(f"📊 Total Rows: {len(df)}")
    print("Preview:")
    print(df[['filename', 'machine_transcript']].head())
else:
    print("❌ Final CSV not found!")


📄 Final File Generated: /kaggle/working/draft_transcriptions_FINAL.csv
📊 Total Rows: 577
Preview:
                               filename  \
0  Chittagonian_JvwgOr-K0vQ_seg0000.wav   
1  Chittagonian_JvwgOr-K0vQ_seg0001.wav   
2  Chittagonian_JvwgOr-K0vQ_seg0002.wav   
3  Chittagonian_JvwgOr-K0vQ_seg0003.wav   
4  Chittagonian_JvwgOr-K0vQ_seg0004.wav   

                                  machine_transcript  
0                                                NaN  
1  নেশিরক্লেন কেনা আদেনা যাকাডি আকাডি পেনি আফারাখ...  
2  तोईले तार फ्रेमी के भुईश्दो तारे अभुयला गरी की...  
3  तुआर भुद्धि। तुआर मतल वचस्तर भुद्धि तोरी। अर भ...  
4  फारा फूती भेशिरे जाना, इते रातो चाने जाइबु गोई...  
